In [144]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np
import folium
import json
import geopandas as gpd
import plotly.express as px
from matplotlib import font_manager, rc
rc('font', family='gulim')

df = pd.read_csv('camping_loc.csv', encoding='UTF8').replace(np.nan, '-').filter(['캠핑(야영)장명', '위도', '경도', '주소'])
df['주소'] = df['주소'].str.lstrip('0123456789-/ ')  #주소 앞에 있는 다른 문자들 제거
df.to_csv('camping_loc')
df
path = 'TL_SCCO_CTPRVNx.json'
geo_data = json.load(open(path, encoding = 'utf-8'))  #json 파일을 로드

## DataFrame

In [146]:
index = ['경기도', '강원도', '충청북도', '충청남도', '경상북도', '경상남도', '전라북도', '전라남도', '제주특별자치도',
         '세종특별자치시', '서울특별시', '광주광역시', '인천광역시', '대전광역시', '대구광역시', '부산광역시', '울산광역시']
loc = pd.DataFrame(index = index, columns = ['개수']).fillna(0) 

# 두 가지로 쓰이는 예외를 처리
cond = [['경기', '경기'], ['강원', '강원'], ['충청북도', '충북'], ['충청남도', '충남'], ['경상북도', '경북'], ['경상남도', '경남'],
       ['전라북도', '전북'], ['전라남도', '전남'], ['제주', '제주'], ['세종', '세종'], ['서울', '서울'], ['광주', '광주'],
       ['인천', '인천'], ['대전', '대전'], ['대구', '대구'], ['부산', '부산'], ['울산', '울산']]

# 반복문으로 해당 지역의 야영자 수를 카운트, dataframe에 저장
for i in range(len(loc.index)):
    loc.iloc[i] = len(df.query("주소.str.startswith('%s') or 주소.str.startswith('%s')" % (cond[i][0], cond[i][1])))
loc

,개수
경기도,534
강원도,475
충청북도,161
충청남도,189
경상북도,248
경상남도,226
전라북도,116
전라남도,137
제주특별자치도,45
세종특별자치시,4


## Folium map

In [147]:
mmap = folium.Map(loc = [36.541, 126.986], zoom_start = 8)  #지도 가져옴

for i in df.index:
    name = df.loc[i, '캠핑(야영)장명']
    lat = df.loc[i, '위도']
    lng = df.loc[i, '경도']
    marker = folium.Marker([lat,lng], popup = name).add_to(mmap)  #[위도,경도] 위치에 따라 지도 위에 표시
    
folium.Choropleth(geo_data = geo_data, data = loc,
                 columns = [loc.index, "개수"],
                 key_on = 'feature.properties.CTP_KOR_NM',
                 fill_color='PuRd').add_to(mmap)
#json 파일을 어디 사용해야 하는지 모르겠어서 지도에 야영장 수에 따라 지역별로 구분하여 색칠하는 데 사용했습니다
mmap

In [148]:
tmp = df.copy()

# 반복문을 돌며 예외처리
for i in range(len(loc.index)):
    a = tmp.query("주소.str.startswith('%s') or 주소.str.startswith('%s')" % (cond[i][0], cond[i][1])).index.tolist()
    for j in reversed(a):
        tmp.drop(j, axis = 0, inplace = True)

tmp  # 주소가 없거나, cond에 없는 주소일경우

,캠핑(야영)장명,위도,경도,주소
437,청도군 자전거공원 캠핑장,35.587208,128.766248,청도군 청도읍 유호리 39-5
503,강산에 글램핑,35.676485,128.984979,청도군 운문면 신원리 390-2
514,펀앤락야영장,35.690537,128.566280,각북면 낙산2길 15-60
606,증산수도계곡캠핑장,35.861259,127.996474,김천시 증산면 수도길 25
616,Camp 1950 by 민들레울,35.878784,127.941044,김천시 대덕면 남김천대로 160
664,산내들 패밀리 어드벤처파크,35.977100,128.015682,김천시 지례면 부항로 109-38
744,군위댐 야영장,36.119373,128.788506,군위군 고로면 화북3길 38-10
787,보경사힐링캠핑장,36.238117,129.342550,포항시 북구 송라면 보경로 307
789,성덕댐 수달 캠핑장,36.243560,128.964395,청송군 안덕면 성덕댐로 1291-70
846,로하스 가족공원 캠핑장,36.455892,127.474642,대청로424번길 200


## 막대그래프 그리기

In [149]:
fig= px.bar(loc, x = loc.index, y = "개수")
fig.update_layout(xaxis = {'title': '지역'})